In [82]:
import pandas as pd

df_raw = pd.read_csv("owid-energy-data.csv")

In [83]:
df_2022 = df[df['year'] == 2022].copy()

In [86]:
import pandas as pd
import plotly.express as px
import numpy as np

df = pd.read_csv("owid-energy-data.csv")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
df = df[df['iso_code'].str.len() == 3].copy()
df_2022 = df[df['year'] == 2022].copy()
df_2022['population_millions'] = df_2022['population'] / 1_000_000

df_2022 = df_2022.dropna(subset=['population_millions', 'fossil_share_energy', 'fossil_fuel_consumption'])

key_countries = ['China', 'India', 'Iceland', 'Norway', 'Saudi Arabia', 'Qatar']
df_2022['label'] = df_2022['country'].where(df_2022['country'].isin(key_countries), '')

df_2022['bubble_size'] = df_2022['fossil_fuel_consumption']

df_2022['marker_size'] = np.where(
    df_2022['country'].isin(key_countries),
    df_2022['bubble_size'] / df_2022['bubble_size'].max() * 60,
    6
)

df_2022['color'] = df_2022['fossil_share_energy']

fig = px.scatter(
    df_2022,
    x='fossil_share_energy',
    y='population_millions',
    size='marker_size',
    color='color',
    text='label',
    color_continuous_scale='Blues',
    labels={
        'color':'Fossil Share (%)',
        'marker_size':'Bubble size: Fossil Fuel Consumption (TWh)'
    },
    hover_name='country'
)

fig.update_traces(
    marker=dict(sizemode='area', line_color='black', line_width=0.5, opacity=0.7),
    textposition='top center'
)

fig.update_layout(
    template='plotly_white',
    font=dict(family='Arial', size=12),
    title={'text':'Global Population vs Fossil Fuel Share of Energy (2022)','x':0.5,'xanchor':'center'},
    xaxis=dict(
        title='Fossil Share Energy (%)',
        dtick=10,
        showgrid=True
    ),
    yaxis=dict(
        title='Population (Millions, log scale)',
        type='log',
        dtick=1
    ),
    legend_title_text='Country Highlight',
    annotations=[
        dict(
            xref='paper', yref='paper', x=1.02, y=0.5, showarrow=False, text='Bubble size: Fossil Fuel Consumption (TWh)', textangle=-90,
            font=dict(size=10)
        ),
        dict(
            xref='paper', yref='paper', x=0, y=-0.15, showarrow=False,
            text='Source: Our World in Data – Energy dataset (based on Energy Institute Statistical Review of World Energy 2025), processed from owid-energy-data.csv. Data for 2022. Population on log scale.',
            font=dict(size=10)
        )
    ]
)

fig.show()

In [89]:
print(df['year'].unique())

[1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003
 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017
 2018 2019 2020 2021 2022]


In [88]:
import pandas as pd
import plotly.express as px

df = pd.read_csv("owid-energy-data.csv")

df = df[
    (df["year"] >= 1990) &
    (df["iso_code"].str.len() == 3) &
    (df["country"] != "World")
]

df = df.dropna(subset=[
    "population", "gdp", "fossil_energy_per_capita", "fossil_fuel_consumption"
])

df["gdp_per_capita"] = df["gdp"] / df["population"]
df["fossil_energy_per_capita_mwh"] = df["fossil_energy_per_capita"] / 1000

selected_countries = [
    "China", "India", "United States", "Norway", "Saudi Arabia",
    "Qatar", "United Arab Emirates", "Germany", "Japan"
]

df["highlight"] = df["country"].apply(
    lambda x: x if x in selected_countries else "Other"
)

fig = px.scatter(
    df,
    x="gdp_per_capita",
    y="fossil_energy_per_capita_mwh",
    size="fossil_fuel_consumption",
    color="highlight",
    hover_name="country",
    animation_frame="year",
    animation_group="country",
    log_x=True,
    log_y=True,
    size_max=55,
    range_x=[500, 150000],
    range_y=[0.5, 200],
    opacity=0.7,
    color_discrete_map={
        "China": "red",
        "India": "orange",
        "United States": "darkblue",
        "Norway": "green",
        "Saudi Arabia": "purple",
        "Qatar": "brown",
        "United Arab Emirates": "pink",
        "Germany": "teal",
        "Japan": "grey",
        "Other": "lightgrey"
    },
    labels={
        "gdp_per_capita": "GDP per capita (USD, log scale)",
        "fossil_energy_per_capita_mwh": "Fossil fuel consumption per capita (MWh/person, log scale)",
        "fossil_fuel_consumption": "Total fossil primary energy (TWh)",
        "year": "Year"
    },
    title="Evolution of Fossil Fuel Dependency vs. Economic Growth (1990–latest)"
)

fig.update_traces(
    mode="markers+lines",
    line=dict(width=2, dash="dot"),
    selector=dict(name=lambda x: x in selected_countries)
)

fig.add_annotation(
    xref="paper", yref="paper",
    x=0.5, y=1.12,
    text="Bubble size: total fossil consumption (TWh). Trails show country paths over time.",
    showarrow=False,
    font=dict(size=12),
    align="center"
)

fig.add_annotation(
    xref="paper", yref="paper",
    x=0.02,
    y=0.02,
    text="Source: Our World in Data – Energy dataset (2025 edition), based on Energy Institute Statistical Review.",
    showarrow=False,
    font=dict(size=8, color="gray"),
    align="left",
    bgcolor="rgba(255,255,255,0.7)",
    bordercolor="lightgray",
    borderwidth=1,
    borderpad=4
)

fig.update_layout(
    template="plotly_white",
    title_x=0.5,
    height=700,
    width=1100,
    legend_title_text="Selected countries",
    margin=dict(t=140, b=140, l=100, r=80),
    xaxis=dict(automargin=True, title_standoff=15),
    yaxis=dict(automargin=True, title_standoff=15),
    sliders=[dict(
        active=0,
        yanchor="top",
        pad=dict(t=25, b=10),
        currentvalue=dict(
            font=dict(size=12),
            prefix="Year: ",
            visible=True,
            xanchor="right"
        ),
        len=0.9,
        x=0.1,
        y=-0.05
    )]
)

fig.update_traces(
    marker=dict(opacity=0.3, size=8),
    selector=dict(name="Other")
)

fig.show()